In [30]:

import pandas as pd
import numpy as np
import os
import openpyxl as xl

import re

import warnings
warnings.filterwarnings('ignore')

from Utilities import FileIO


# Load Files

In [31]:
df = FileIO.importFile("transformFinished_merged.csv")
df.head()

,product_id,product,mail_class,fy,volume,revenue,attributable_cost,volume_variable_cost,product_specific_cost,rev_pc,attributable_cost_pc,cont_pc,cost_coverage,weight,weight_pc
0,3,Single Piece Letters,First Class,2008,3.350971e+10,1.435352e+10,8.443076e+09,8.434323e+09,8753000.0,0.428339,0.251959,0.176380,1.700035,1.000329e+09,0.477631
1,4,Single Piece Cards,First Class,2008,1.845860e+09,5.004896e+08,4.465088e+08,4.460268e+08,482000.0,0.271142,0.241897,0.029244,1.120895,1.183008e+07,0.102544
2,5,Total Single Piece Letters and Cards,First Class,2008,3.535557e+10,1.485401e+10,8.889584e+09,8.880349e+09,9235000.0,0.420132,0.251434,0.168698,1.670946,1.012159e+09,0.458048
3,8,Presort Letters,First Class,2008,4.837987e+10,1.632780e+10,5.441668e+09,5.429030e+09,12638000.0,0.337492,0.112478,0.225014,3.000515,2.174874e+09,0.719266
4,9,Presort Cards,First Class,2008,3.555997e+09,7.322368e+08,2.822705e+08,2.813415e+08,929000.0,0.205916,0.079379,0.126537,2.594096,2.896925e+07,0.130345


# Handle Calculated Columns

In [32]:
df.columns

Index(['product_id', 'product', 'mail_class', 'fy', 'volume', 'revenue',
       'attributable_cost', 'volume_variable_cost', 'product_specific_cost',
       'rev_pc', 'attributable_cost_pc', 'cont_pc', 'cost_coverage', 'weight',
       'weight_pc'],
      dtype='object')

In [33]:
coreCols = ['product_id', 'fy', 'volume', 'revenue',
       'attributable_cost', 'volume_variable_cost', 'product_specific_cost',
       'weight']

def normalizeFrame(frame):
    normalizedDf = frame[coreCols]
    return normalizedDf

df_normalized = normalizeFrame(df)

In [34]:
def calculateCols(frame):

    def createDivCol(num, denom, rezName):
        percision = 8
        frame[rezName] = np.where(
            frame[denom] ==0 | frame[denom].isnull(),
            np.nan,
            round(frame[num]/frame[denom], percision)
        )

    def createSubCol(num, denom, rezName):
        percision = 8
        frame[rezName] = np.where(
            frame[num].isnull()| frame[denom].isnull(),
            np.nan,
            round(frame[num] - frame[denom], percision)
        )
        
    createDivCol('revenue','volume','rev_pc')
    createDivCol('attributable_cost','volume','attributable_cost_pc')

    createSubCol("revenue", "attributable_cost", "contribution")


    createDivCol('contribution','volume','cont_pc')

    createDivCol('revenue','attributable_cost','cost_coverage')
    createDivCol('weight','volume','weight_pc')
        
    return frame

df_full = calculateCols(df)
# df_full

In [35]:
def reportCalcDiffs(calcCol):
    temp = df_full.merge(df, how="left", on=["fy", "product_id"])
    xCol, yCol = calcCol+'_x',  calcCol+'_y'
    temp['diff'] = temp[xCol] - temp[yCol]
    filt = temp[(temp['diff']!=0) & (temp['diff'].notnull())]
    # print(filt)
    if len(filt)>0:
        print(filt)


for col in ['rev_pc', 'attributable_cost_pc', 'cont_pc', 'cost_coverage',  'weight_pc']:
    reportCalcDiffs(col)

# Round Columns

In [36]:
def roundCols(frame):
    for col in ['revenue','volume','attributable_cost', 'volume_variable_cost', 'product_specific_cost']:
        frame[col] = frame[col].round(0)

    return frame


df_normalized = roundCols(df_normalized )
df_full = roundCols(df_full )


# Upload Finished Files

In [37]:

FileIO.exportFile(df_normalized, "loadFinished_normalized.csv")
FileIO.exportFile(df_full, "loadFinished_notNormalized.csv")


 # Garbage

In [38]:
# df = df_normalized.copy()

# # df[df['volume']].sort_values(by="volume")
# def sortCol(col):
#     return df[df[col]!=0].sort_values(by=col).head(20)

# # sortCol("volume")
# # sortCol("revenue")
# # sortCol("attributable_cost")
# # sortCol("weight")
# # df['volume'] = df['volume'].astype(int)

# def applyInteger(row):
#     # if row==np.nan:
#     if pd.isna(row):
#         return row
#     return int(row)

# df['volume'] = df['volume'].apply(lambda row: applyInteger(row))

# sortCol("volume")

In [39]:
df_db = pd.read_excel("./garbage/cra from db.xlsx")
# df_db.head()
df = df_normalized.copy()

In [40]:
toMerge = df_db
# toMerge.rename(columns={"revneue":"rev_db"})
# toMerge.columns
# keepCols =['product', 'product_id', 'fy', 'row_id', 'mail_class', 'volume',
#        'revenue', 'attributable_cost']

for col in [ 'volume','revenue', 'attributable_cost']:
    df.rename(columns={col: col+"_orig"}, inplace=True)


# toMerge = toMerge[keepCols]

mf = df.merge(toMerge, how="left", on=['fy','product_id'])

for col in ['volume','revenue', 'attributable_cost']:
    newColName = col +"_diff"
    mf[newColName] = mf[col+"_orig"] -  mf[col]

mf.head()

,product_id,fy,volume_orig,revenue_orig,attributable_cost_orig,volume_variable_cost_x,product_specific_cost_x,weight_x,volume,revenue,attributable_cost,volume_variable_cost_y,product_specific_cost_y,weight_y,volume_diff,revenue_diff,attributable_cost_diff
0,3,2008,3.350971e+10,1.435352e+10,8.443076e+09,8.434323e+09,8753000.0,1.000329e+09,3.350971e+10,1.435352e+10,8.443076e+09,8.434323e+09,8753000.0,1.000329e+09,0.0,0.0,0.0
1,4,2008,1.845860e+09,5.004896e+08,4.465088e+08,4.460268e+08,482000.0,1.183008e+07,1.845860e+09,5.004896e+08,4.465088e+08,4.460268e+08,482000.0,1.183008e+07,0.0,0.0,0.0
2,5,2008,3.535557e+10,1.485401e+10,8.889584e+09,8.880349e+09,9235000.0,1.012159e+09,3.535557e+10,1.485401e+10,8.889584e+09,8.880349e+09,9235000.0,1.012159e+09,0.0,0.0,0.0
3,8,2008,4.837987e+10,1.632780e+10,5.441668e+09,5.429030e+09,12638000.0,2.174874e+09,4.837987e+10,1.632780e+10,5.441668e+09,5.429030e+09,12638000.0,2.174874e+09,0.0,0.0,0.0
4,9,2008,3.555997e+09,7.322368e+08,2.822705e+08,2.813415e+08,929000.0,2.896925e+07,3.555997e+09,7.322368e+08,2.822705e+08,2.813415e+08,929000.0,2.896925e+07,0.0,0.0,0.0


In [41]:



# for col in [c for c in mf.columns if "diff" in c]:
#     print(mf[col].value_counts())

volume_diff
0.0    779
Name: count, dtype: int64
revenue_diff
0.0    970
Name: count, dtype: int64
attributable_cost_diff
0.0    905
Name: count, dtype: int64
